In [ ]:
#default_exp experiments

# Experiments

> Experiments to be run as CLI scripts (too long to run in a Jupyter session)

In [ ]:
#export
import papermill as pm
import numpy as np
from fastcore.all import *

In [ ]:
import wandb

### Analyse the impact of the number of points on the performance of the motion classifier

The results will be logged in a run group in wandb, and from there the visualizations will be made

In [ ]:
#export experiments
@call_parse
def x_sequence_length(min_rp:Param("Minimum resampling period", int)=1,
                      max_rp:Param("Maximum resampling period", int)=10,
                      rep_times:Param("Number of times ru run per rp", int)=10,
                      nbs_folder:Param("Notebook folder path", str)='.'):
    for x in range(min_rp, max_rp+1):
        for _ in range(rep_times):
            pm.execute_notebook(input_path=f'{Path(nbs_folder)}/_motion_classification_time_series.ipynb',
                                output_path='--',
                                parameters=dict(
                                    use_wandb=True,
                                    wandb_group='x_sequence_length',
                                    resampling_period=x
                                ))

In [ ]:
%%capture
x_sequence_length(max_rp=2, rep_times=1)

### Analyse how the inclusion of more and more data helps to predict better the motions of the next epsilon (x_next_epsilon)

In [ ]:
#export experiments
@call_parse
def x_next_epsilon(nbs_folder:Param("Notebook folder path", str)='.'):
    base_path = Path.home()
    ds_path = base_path/'data/fli_rotnum/model_1'
    fnames = L(
        (ds_path/'eps=0.021/Poincare.plt', ds_path/'eps=0.021/index_fli_rotnum_birav.plt'),
        (ds_path/'eps=0.022/Poincare.plt', ds_path/'eps=0.022/index_fli_rotnum_birav.plt'), 
        (ds_path/'eps=0.023/Poincare.plt', ds_path/'eps=0.023/index_fli_rotnum_birav.plt'), 
        (ds_path/'eps=0.024/Poincare.plt', ds_path/'eps=0.024/index_fli_rotnum_birav.plt'), 
        (ds_path/'eps=0.025/Poincare.plt', ds_path/'eps=0.025/index_fli_rotnum_birav.plt'), 
    )
    for i in range(2, len(fnames)+1):
        x_input = L(fnames[:i])
        val_dataset = x_input[-1][0].parent.name
        print(val_dataset)
        pm.execute_notebook(input_path=f'{Path(nbs_folder)}/_motion_classification_time_series.ipynb',
                            output_path='./__.ipynb',
                            parameters=dict(
                                use_wandb=True,
                                wandb_group='x_next_epsilon',
                                val_dataset = val_dataset,
                                fnames_poinc_map=list(x_input.itemgot(0).map(str)),
                                fnames_index=list(x_input.itemgot(1).map(str))
                            ))

In [ ]:
x_next_epsilon()

eps=0.022


PapermillExecutionError: 
---------------------------------------------------------------------------
Exception encountered at "In [22]":
---------------------------------------------------------------------------
AssertionError                            Traceback (most recent call last)
<ipython-input-22-83641df2a7df> in <module>
----> 1 ds = Datasets(items=data.get_items(), tfms=tfms, splits=splits)

/usr/local/lib/python3.6/dist-packages/fastai/data/core.py in __init__(self, items, tfms, tls, n_inp, dl_type, **kwargs)
    308     def __init__(self, items=None, tfms=None, tls=None, n_inp=None, dl_type=None, **kwargs):
    309         super().__init__(dl_type=dl_type)
--> 310         self.tls = L(tls if tls else [TfmdLists(items, t, **kwargs) for t in L(ifnone(tfms,[None]))])
    311         self.n_inp = ifnone(n_inp, max(1, len(self.tls)-1))
    312 

/usr/local/lib/python3.6/dist-packages/fastai/data/core.py in <listcomp>(.0)
    308     def __init__(self, items=None, tfms=None, tl

## Try different validation datasets

In [ ]:
x_val_datasets = {
    'name': 'x_val_datasets',
    'description': 'Test different validation datasets,  \
                    keeping the training set constant',
    'method': 'grid',
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'val_ds': {
            'distribution': 'categorical',
            'values': ['eps=0.021', 'eps=0.022', 'eps=0.023', 'eps=0.024', 'eps=0.025']
        }
    }
}

## x_next_epsilon

In [ ]:
x_next_epsilon = {
    'name': 'x_next_epsilon',
    'description': 'keeping the training set constant to eps=0.021, validate on incremental \
    values of epsilon from eps=0.021 (training=validaton) to eps=0.025, to see how the distance \
    to the training parameters affect performance.',
    'method': 'grid',
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'val_ds': {
            'distribution': 'categorical',
            'values': ['eps=0.021', 'eps=0.022', 'eps=0.023', 'eps=0.024', 'eps=0.025']
        }
    }
}

In [ ]:
sweep_id=wandb.sweep(x_next_epsilon, project='mlchaos')

In [ ]:
wandb.agent(sweep_id,
            function=partial(pm.execute_notebook, 
                             input_path='./_motion_classification_time_series.ipynb', 
                             output_path='__.ipynb', 
                             parameters=dict(
                                 fnames_poinc_map=['/home/user/data/fli_rotnum/model_1/eps=0.021/Poincare.plt'],
                                 fnames_index=['/home/user/data/fli_rotnum/model_1/eps=0.021/index_fli_rotnum_birav.plt'], 
                                 resampling_period=2, 
                                 use_wandb=True)
                            ))

## Analyse the effect of the labelling method

In [ ]:
%nbdev_export experiments
@call_parse
def x_labelling_method(rep_times:Param("Number of times to run per experiment", int)=10,
                      nbs_folder:Param("Notebook folder path", str)='.'):
    for lbl_method in ['fli', 'freq_ana', 'birkhoff_avg']:
        for _ in range(rep_times):
            pm.execute_notebook(input_path=f'{Path(nbs_folder)}/_motion_classification_time_series.ipynb',
                                output_path='--',
                                parameters=dict(
                                    use_wandb=True,
                                    wandb_group='x_labelling_method',
                                    labelling_method=lbl_method
                                ))